In [17]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os
import json
import requests
import re

In [18]:
# Define la URL de tu endpoint en FastAPI
url = "http://127.0.0.1:8008/process_pdf"

# Define los datos que quieres enviar en la solicitud
data = {
  "items": [
    "Quiero que hagas un resumen del texto teniendo en cuenta que se trata de una reunion entre un cliente y un proveedor, y que de la reunion tiene por objetivo relevar informacion de un sistema que se encuentra en funcionamiento. Para generar el resumen debes incorporar el tiempo de inicio. Te muestro un schema de ejemplo para dos intervalos de tiempo puede tener mas o menos. Schema: {'t1':'tiempo de inicio utilizar formato HH:MM:SS', 'resumen1':resumen del texto, t2':'tiempo de inicio utilizar formato HH:MM:SS', 'resumen2':resumen del texto}"
  ]
}

# Realiza la solicitud POST con los datos
response = requests.post(url, json=data)

# Muestra la respuesta
print(response.json())

None


In [19]:
import mysql.connector
from mysql.connector import Error
import os
import pandas as pd

def convert_to_dataframe(results):
    # Convierte los resultados a una lista de diccionarios
    data = []
    for result in results:
        try:
            json_str = result[0].replace("'", "\"")  # Reemplaza comillas simples por dobles
            json_dict = json.loads(json_str)
            data.append(json_dict)
        except json.JSONDecodeError:
            print(f"Error al decodificar JSON: {result[0]}")
            continue
    
    # Crea un DataFrame de pandas a partir de la lista de diccionarios
    df = pd.DataFrame(data)
    return df



def read_processed_files(pdf_file):
    try:
        # Configuración de la conexión
        connection = mysql.connector.connect(
            host='mysql',  # Usamos el nombre del servicio definido en docker-compose.yml
            user='root',  # Usuario de MySQL
            password=os.getenv('MYSQL_ROOT_PASSWORD'),  # Contraseña del usuario de MySQL
            database=os.getenv('MYSQL_DATABASE')  # Nombre de la base de datos
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"Conectado a MySQL Server versión {db_info}")
            cursor = connection.cursor()
            cursor.execute("SELECT r FROM results WHERE pdf_name = %s", (str(pdf_file),))
            file_result = cursor.fetchall()
            # Si no necesitas más resultados, asegúrate de descartar los no leídos
            if cursor.nextset():
                cursor.fetchall()  # Descarta los resultados adicionales
            return file_result

    except Error as e:
        print("Error durante la conexión o la consulta", e)
        return 'error'

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión a MySQL cerrada")

# Ejemplo de uso
pdf_file ='./pdfs/2005.pdf'
result = read_processed_files(pdf_file)
if result != 'error':
    df = convert_to_dataframe(result)
else:
    print("Error al obtener los resultados de la base de datos")
df.head()


Conectado a MySQL Server versión 5.7.44
Conexión a MySQL cerrada
Error al decodificar JSON: '{
  "t1": "37:32",
  "resumen1": "Se discute sobre el rechazo de cobertura y el regreso del trabajador. Sergio y Mapply confirman que todo está claro.",
  "t2": "37:54",
  "resumen2": "Mapply pregunta si hay algo más que discutir sobre rechazos o registros administrativos. José confirma que no falta nada. Mapply repasa que César enviará la lógica de los criterios y menciona que están trabajando en una versión que se incrementará en reuniones futuras. Sergio informa que están subiendo información en un Word y que avisarán cuando esté lista para revisión.",
  "t3": "39:54",
  "resumen3": "Vale menciona que revisará los datos de la denuncia subidos por César y Ceci. Prestadores comenta que hubo un problema al publicar el archivo en Teams, pero lo enviaron por correo. Vale confirma que revisará el archivo del correo.",
  "t4": "40:45",
  "resumen4": "César tomará nota de los puntos de dolor mencion

,t1,resumen1,t2,resumen2,t3,resumen3,t4,resumen4,t5,resumen5,t6,resumen6,t7,resumen7,t8,resumen8
0,00:00,Los participantes de la reunión se saludan y c...,03:15,El objetivo de la reunión es comenzar a revisa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01:53,Los participantes discuten sobre la asistencia...,03:15,"Mapply I. explica el objetivo de la reunión, q...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03:46,Mapply I. y Cesar R. discuten sobre la función...,04:26,Cesar R. le pide a José que explique las funci...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,05:54,Cesar R. y Jose R. discuten sobre el análisis ...,06:53,Jose R. explica el proceso de análisis de info...,07:24,Cesar R. pregunta sobre los criterios de inves...,08:32,Sergio G. sugiere automatizar la asignación de...,10:05,"Jose R. concluye explicando que, una vez recib...",NaN,NaN,NaN,NaN,NaN,NaN
4,08:32:00,Jose R. menciona que no tiene información sobr...,08:52:00,Sergio G. sugiere automatizar la asignación de...,09:35:00,Sergio G. propone hacer la asignación automáti...,10:05:00,Jose R. explica que una vez recibidos los info...,10:33:00,César R. menciona la suspensión de plazos para...,11:12:00,César R. detalla el proceso de suspensión de p...,12:03:00,Mapply I. pregunta si se informa sobre el esta...,NaN,NaN


In [20]:
df_melted_t = pd.melt(df, value_vars=[col for col in df.columns if col.startswith('t')], var_name='variable', value_name='t')
df_melted_resumen = pd.melt(df, value_vars=[col for col in df.columns if col.startswith('resumen')], var_name='variable', value_name='resumen')

# Eliminar la columna 'variable' ya que no es necesaria
df_melted_t.drop('variable', axis=1, inplace=True)
df_melted_resumen.drop('variable', axis=1, inplace=True)

# Concatenar los DataFrames derretidos (melted)
df_total = pd.concat([df_melted_t, df_melted_resumen], axis=1)

# Mostrar el DataFrame resultante
df_total.head()

,t,resumen
0,00:00,Los participantes de la reunión se saludan y c...
1,01:53,Los participantes discuten sobre la asistencia...
2,03:46,Mapply I. y Cesar R. discuten sobre la función...
3,05:54,Cesar R. y Jose R. discuten sobre el análisis ...
4,08:32:00,Jose R. menciona que no tiene información sobr...


In [21]:
df_total.to_excel('./punteo.xlsx', index=False)
df_total = df_total.dropna(subset=['resumen'])
texto_resumen = df_total['resumen'].to_string(index=False)
print(texto_resumen)

Los participantes de la reunión se saludan y co...
Los participantes discuten sobre la asistencia ...
Mapply I. y Cesar R. discuten sobre la función ...
Cesar R. y Jose R. discuten sobre el análisis d...
Jose R. menciona que no tiene información sobre...
Sergio G. menciona algo relacionado con el tema...
Jose R. explica que si hay una investigación pe...
Cesar R. menciona la importancia de obtener inf...
Vale A. menciona un incidente reciente en San C...
La conversación comienza con una discusión sobr...
Vale A. sugiere automatizar el envío de un códi...
Cesar R. y Vale A. discuten sobre la identifica...
Jose R. explica el proceso de verificación de d...
Víctor G.S. menciona el tema de la nómina inici...
Cesar R. menciona un problema con un cliente nu...
Se discute sobre la rescisión de contratos y có...
Sergio G. consulta sobre el proceso de rechazo ...
Vale A. menciona que revisará los datos de la d...
La reunión comienza con la confirmación de que ...
El objetivo de la reunión es co

In [22]:
def generate_abstract(text):
    API_KEY = os.getenv('API_KEY')
    API_URL = os.getenv('API_URL')
    MODEL = os.getenv('MODEL')
    TEMPERATURE = float(os.getenv('TEMPERATURE'))
    
    
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json',
    }
    #aca jugar con los modelos 
    # data = {
    #     'model': MODEL,
    #     'messages': [
    #         {"role": "system", "content": f"Eres un asistente de IA util. Tu objetivo principal es generar posibles preguntas para el texto que se te proporciona como contexto. Estas preguntas deben tener sentido en el contexto del texto y deben estar pensadas para elaborar un informe.El texto que se te provee es un punteo de reuniones con clientes para relevar un sistema. Texto: {text},
    #         {"role": "user", "content": f"usa el texto proporcionado para generar la respuesta"}
    #     ],
    #     "options": {
    #         "temperature": TEMPERATURE,
    #         "num_predict":1000    #maxima cantidad de tokens ollama
    #     },
    #     "format":"json"

    # }

    data = {
        'model': MODEL,
        'temperature': TEMPERATURE,
        'messages': [
            {"role": "system", "content": f"Eres un asistente de IA util. Tu objetivo principal es generar un breve resumen del texto de una reunion. Texto: {text}"},
            {"role": "user", "content": f"Genera en no mas de 40 palabras"}
                    ]
            }
    response = requests.post(API_URL, headers=headers, json=data)
    if response.status_code == 200:
        response_json = response.json()
        # Asumimos que la última respuesta del modelo es la respuesta a la pregunta
        return response_json['choices'][0]['message']['content']
    else:
        return response_json

abstract=generate_abstract(texto_resumen)
print(abstract)

En la reunión se discutieron temas como la automatización de procesos, la verificación de datos, la importancia de la investigación de fraudes, y la necesidad de mejorar la comunicación y gestión de incidentes y siniestros.


In [23]:
schema={
  "tema1": {
    "titulo": "Titulo del tema",
    "preguntas": {
        "pregunta1": "pregunta",
        "pregunta2": "pregunta"
  }
  },
  "tema2": {
    "titulo": "Titulo del tema",
    "preguntas": 
      {
        "pregunta1": "pregunta",
        "pregunta2": "pregunta"
      }
  }
}


def generate_questions(text,n):
    API_KEY = os.getenv('API_KEY')
    API_URL = os.getenv('API_URL')
    MODEL = os.getenv('MODEL')
    TEMPERATURE = float(os.getenv('TEMPERATURE'))
    
    
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json',
    }
    #aca jugar con los modelos 
    # data = {
    #     'model': MODEL,
    #     'messages': [
    #         {"role": "system", "content": f"Eres un asistente de IA util. Tu objetivo principal es generar posibles preguntas para el texto que se te proporciona como contexto. Estas preguntas deben tener sentido en el contexto del texto y deben estar pensadas para elaborar un informe.El texto que se te provee es un punteo de reuniones con clientes para relevar un sistema. Texto: {text},
    #         {"role": "user", "content": f"usa el texto proporcionado para generar la respuesta"}
    #     ],
    #     "options": {
    #         "temperature": TEMPERATURE,
    #         "num_predict":1000    #maxima cantidad de tokens ollama
    #     },
    #     "format":"json"

    # }

    data = {
        'model': MODEL,
        'temperature': TEMPERATURE,
        'messages': [
            {"role": "system", "content": f"Eres un asistente de IA util. Tu objetivo principal es generar posibles preguntas para el texto que se te proporciona como contexto. Estas preguntas deben tener sentido en el contexto del texto y deben estar pensadas para elaborar un informe.El texto que se te provee es un punteo de reuniones con clientes para relevar un sistema. Texto: {text} Responde en formato json utilizando unicamente el siguiente schema: {schema}. no agregues triple comilla ni nada adicional, la respuesta debe ser el schema json pedido que podra contaner mas o menos elmentos dependiendo de los datos, no olvides incluir la key tema con el numero dentro del json"},
            {"role": "user", "content": f"Genera al menos {str(n)} preguntas (pueden ser mas) utilizando como contexto unicamente el texto proporcionado, dividelas por temas con titulos representativos de los procesos teniendo en cuenta que se trata de generar un reporte de la reunion. . Verifica que las preguntas son distintas y no generan respuestas similares"}
                    ]
            }
    response = requests.post(API_URL, headers=headers, json=data)
    if response.status_code == 200:
        response_json = response.json()
        # Asumimos que la última respuesta del modelo es la respuesta a la pregunta
        return response_json['choices'][0]['message']['content']
    else:
        return response_json

In [24]:
questions=generate_questions(texto_resumen,3)
questions = re.sub(r'```json.*?```', '', questions, flags=re.DOTALL)
questions = re.sub(r'```.*?```', '', questions, flags=re.DOTALL)
print(questions)

{
  "tema1": {
    "titulo": "Automatización de Procesos",
    "preguntas": {
      "pregunta1": "¿Qué beneficios se esperan al automatizar el envío de códigos?",
      "pregunta2": "¿Cómo se podría implementar la automatización de la asignación de siniestros?",
      "pregunta3": "¿Qué pasos se deben seguir para automatizar la asignación de tareas en el sistema?"
    }
  },
  "tema2": {
    "titulo": "Análisis de Información",
    "preguntas": {
      "pregunta1": "¿Qué métodos se utilizan actualmente para el análisis de información?",
      "pregunta2": "¿Cómo se puede mejorar el proceso de verificación de datos?",
      "pregunta3": "¿Qué información adicional es necesaria para realizar un análisis más completo?"
    }
  },
  "tema3": {
    "titulo": "Gestión de Incidentes y Siniestros",
    "preguntas": {
      "pregunta1": "¿Cuál es el procedimiento actual para rechazar una denuncia?",
      "pregunta2": "¿Qué criterios se utilizan para la investigación de siniestros?",
      "pre

In [25]:

questions = re.sub(r"'", r'"', questions)
data = json.loads(questions)
data['tema1']['preguntas']['pregunta1']
# Iterar a través del JSON y mostrar el título, las preguntas y las respuestas


'¿Qué beneficios se esperan al automatizar el envío de códigos?'

In [26]:
filas = []
for tema, contenido in data.items():
    titulo = contenido["titulo"]
    for key, pregunta in contenido["preguntas"].items():
        filas.append({"Tema": tema, "Titulo": titulo, "PreguntaID": key, "Pregunta": pregunta})

# Crear un DataFrame de pandas
df = pd.DataFrame(filas)

In [27]:
df.head()

,Tema,Titulo,PreguntaID,Pregunta
0,tema1,Automatización de Procesos,pregunta1,¿Qué beneficios se esperan al automatizar el e...
1,tema1,Automatización de Procesos,pregunta2,¿Cómo se podría implementar la automatización ...
2,tema1,Automatización de Procesos,pregunta3,¿Qué pasos se deben seguir para automatizar la...
3,tema2,Análisis de Información,pregunta1,¿Qué métodos se utilizan actualmente para el a...
4,tema2,Análisis de Información,pregunta2,¿Cómo se puede mejorar el proceso de verificac...


In [28]:
df['String'] = df.apply(lambda row: f"Teniendo en cuenta el tema {row['Titulo']} quiero saber {row['Pregunta']} para responder piensa que estas elaborando un informe sobre el tema. No repitas la pregunta en tu respuesta, solo responde.", axis=1)

# Mostrar el DataFrame con la nueva columna
for index, row in df.iterrows():
    print(row['String'])

Teniendo en cuenta el tema Automatización de Procesos quiero saber ¿Qué beneficios se esperan al automatizar el envío de códigos? para responder piensa que estas elaborando un informe sobre el tema. No repitas la pregunta en tu respuesta, solo responde.
Teniendo en cuenta el tema Automatización de Procesos quiero saber ¿Cómo se podría implementar la automatización de la asignación de siniestros? para responder piensa que estas elaborando un informe sobre el tema. No repitas la pregunta en tu respuesta, solo responde.
Teniendo en cuenta el tema Automatización de Procesos quiero saber ¿Qué pasos se deben seguir para automatizar la asignación de tareas en el sistema? para responder piensa que estas elaborando un informe sobre el tema. No repitas la pregunta en tu respuesta, solo responde.
Teniendo en cuenta el tema Análisis de Información quiero saber ¿Qué métodos se utilizan actualmente para el análisis de información? para responder piensa que estas elaborando un informe sobre el tema. 

In [29]:
import os
import docx
from datetime import datetime
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

# Create a document
doc = docx.Document()

# Add report title and date
title = doc.add_heading(level=1)
title_run = title.add_run("Reporte")
title_run.font.name = 'Times New Roman'
title_run.font.size = docx.shared.Pt(16)
title_run.bold = True

date = doc.add_paragraph()
date_run = date.add_run(datetime.now().strftime("%Y-%m-%d"))
date_run.font.name = 'Times New Roman'
date_run.font.size = docx.shared.Pt(12)

doc.add_paragraph()

abstract_title = doc.add_heading(level=2)
abstract_title_run = abstract_title.add_run("Resumen de la reunion")
abstract_title_run.font.name = 'Times New Roman'
abstract_title_run.font.size = docx.shared.Pt(12)
abstract_title_run.bold = True



# Add the abstract after the title and date
abstract_paragraph = doc.add_paragraph()
abstract_paragraph.alignment = docx.enum.text.WD_ALIGN_PARAGRAPH.CENTER
abstract_run = abstract_paragraph.add_run(abstract)
abstract_run.font.name = 'Times New Roman'
abstract_run.font.size = docx.shared.Pt(10)
abstract_run.bold = True

# Add some space after the abstract
doc.add_paragraph()

# Add table of contents
toc_title = doc.add_heading('Índice', level=2)
toc_title_run = toc_title.add_run('Índice')
toc_title_run.font.name = 'Times New Roman'
toc_title_run.font.size = docx.shared.Pt(14)
toc_title_run.bold = True

toc = doc.add_paragraph()
toc.style = 'Normal'

OPENAI_API_KEY = os.getenv('API_KEY')  # add your OpenAI API Key

# for this example I used Alphabet Inc 10-K Report 2022
# https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm
NAMESPACE = "Reunion20"
DOC_PATH = "../pdfs/2005.pdf"
CHROMA_PATH = f"./report_db/{NAMESPACE}"

# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

# ----- Retrieval and Generation Process -----

last_title = ''
for index, row in df.iterrows():
    if last_title != row['Titulo']:
        # Add title to table of contents
        toc_run = toc.add_run(row['Titulo'])
        toc_run.font.name = 'Times New Roman'
        toc_run.font.size = docx.shared.Pt(12)
        toc_run.bold = True
        toc.add_run("\n")
        
        # Add title paragraph
        p = doc.add_paragraph()
        p.paragraph_format.line_spacing = 1
        p.paragraph_format.space_after = docx.shared.Pt(24)  # Add space after the title
        
        run = p.add_run(row['Titulo'])
        run.bold = True
        run.font.name = 'Times New Roman'
        run.font.size = docx.shared.Pt(12)
        
        last_title = row['Titulo']
    
    # Add text paragraph
    p = doc.add_paragraph()
    p.paragraph_format.line_spacing = 1
    p.paragraph_format.space_before = docx.shared.Pt(24)  # Add space before the text
    p.paragraph_format.space_after = docx.shared.Pt(24)   # Add space after the text
    
    # Retrieve context - top 5 most relevant (closest) chunks to the query vector
    query = row['String']
    docs_chroma = db_chroma.similarity_search_with_score(query, k=7)
 
    
    # Generate an answer based on given user query and retrieved context information
    context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

    print(context_text)
    
    # You can use a prompt template
    PROMPT_TEMPLATE = f"""
    Responde la pregunta basandote unicamente en el siguiente contexto:
    {context_text}
    Responde la pregunta basandote en el contexto, pon ejemplos basandote en el contexto, y si hay nombre de las personas involucradas usalos. pregunta: {query}. Crea una respuesta detallada con un texto amigable y que sea simple de leer.
    No justifiques tus respuestas.
    No entregues informacion no mencionada en el contexto.
    No digas de acuerdo al contexto proporcionado, o en el contexto proporcionado o similar.
    Utiliza la pregunta para introducir el tema que se esta hablando y luego generar la respuesta.
    """
    
    # Load retrieved context and user query in the prompt template
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query)
    
    # Call LLM model to generate the answer based on the given context and query
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=os.getenv('MODEL'), temperature=float(os.getenv('TEMPERATURE')))
    response_text = model.predict(prompt)
    print(response_text)
    
    run = p.add_run(response_text)
    run.font.name = 'Times New Roman'
    run.font.size = docx.shared.Pt(10)

# Add page numbers in the footer
section = doc.sections[0]
footer = section.footer
footer_paragraph = footer.paragraphs[0]
footer_paragraph.alignment = docx.enum.text.WD_ALIGN_PARAGRAPH.CENTER
run = footer_paragraph.add_run()
fldChar = OxmlElement('w:fldChar')
fldChar.set(qn('w:fldCharType'), 'begin')
instrText = OxmlElement('w:instrText')
instrText.text = "PAGE"
fldChar2 = OxmlElement('w:fldChar')
fldChar2.set(qn('w:fldCharType'), 'end')
run._r.append(fldChar)
run._r.append(instrText)
run._r.append(fldChar2)

# Save the document
doc.save('reporte_mejorado_mapply.docx')

22:02 Vale A.: No, en lo que se me está ocurriendo, que no sé si es viable José 
que por ahí cuando vos asignas un estudio mandarle un código al estudio 
automáticamente también digo esto todo automatizado y que ese mismo código 
se lo mandemos al cliente y entonces haces transparente que cuando se pida se 
te va a pedir o con este código con esta palabra clave o con esto con algo que se 
pueda generar automáticamente entonces ahí como que bueno decís listo como

22:02 Vale A.: No, en lo que se me está ocurriendo, que no sé si es viable José 
que por ahí cuando vos asignas un estudio mandarle un código al estudio 
automáticamente también digo esto todo automatizado y que ese mismo código 
se lo mandemos al cliente y entonces haces transparente que cuando se pida se 
te va a pedir o con este código con esta palabra clave o con esto con algo que se 
pueda generar automáticamente entonces ahí como que bueno decís listo como

Jose le avisan automático o le escriben aparte un mail?
23:00 Jo

AttributeError: 'Run' object has no attribute 'add_hyperlink'